In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
print(nb_dir)
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
# autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
autolib_dir = '/'.join(nb_dir.split('/')[:-1])
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)

print(sys.path)

%load_ext autoreload
%autoreload 2

/home/kai/data/shiyi/AlphaBoosting/automl/automl_app
['', '/home/kai/anaconda3/envs/tf_gpu/lib/python35.zip', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/plat-linux', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/lib-dynload', '/home/kai/.local/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/IPython/extensions', '/home/kai/.ipython', '/home/kai/data/shiyi/AlphaBoosting/automl']


In [2]:
from automl_libs import base_layer_utils, stacknet
from automl_libs import feature_engineering as fe
from automl_libs import encoding, kernels, nn_libs, utils 

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
categorical_features = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance'] 

In [4]:
import itertools
#params = {'split_col': 't', 'coefficient': 10, 'n': 2, 'fillna': 22}
def get_features_to_gen(function_list, low, high):
    features_to_gen = []
    for function in function_list:
        for i in range(low, high+1):
            for combine in itertools.combinations(categorical_features, i):
                if function.__name__ == 'count_std_over_mean':
                    features_to_gen.append({'params': {'coefficient': 10}, 'function': function, 'feature_cols': list(combine)})
                else:
                    features_to_gen.append({'params': {}, 'function': function, 'feature_cols': list(combine)})
                
    return features_to_gen

# features_to_gen = get_features_to_gen([fe.count, fe.unique_count, 
#                                        fe.cumulative_count, 
#                                        fe.reverse_cumulative_count, 
#                                        fe.variance, fe.count_std_over_mean], 
#                                       2, 2)# len(categorical_features))
# features_to_gen = get_features_to_gen([fe.count, fe.unique_count, fe.cumulative_count])
features_to_gen = get_features_to_gen([fe.count], 2, 2)#len(categorical_features))

features_to_gen 

[{'feature_cols': ['Month', 'DayofMonth'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DepTime'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'UniqueCarrier'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'Origin'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'Dest'],
  'function': <function automl_libs.feature_engineering.co

In [5]:
len(features_to_gen)

28

In [6]:
project_path = './' 

In [7]:
from automl_app import logger_config
import logging
logger_config.config(project_path+'project.log', file_loglevel=logging.INFO)

In [8]:
def params_gen(model='lgb'):
    import time
    import numpy as np
    seed = int(time.time()* 1000000) % 45234634
    np.random.seed(seed)
    if model == 'svc':
        params = {
            'C': 0.911#np.random.rand(),
#             'metric': 'roc_auc'
        }
    elif model == 'logreg':
        params = {
            'penalty': np.random.choice(['l2']),
            'dual': np.random.choice([True, False]),
            'C': 1,#np.random.rand(),
            'metric': 'roc_auc'
        }
    elif model == 'lgb':
        params = {    
            'objective': 'binary',
            'boosting': 'gbdt',
            'num_boost_round': 10, # ignored in params. extract it and put it in input arguments in train or cv explicitly
                                   # seems to work fine as the upper limit when combined with early_stopping_round
            'learning_rate': np.random.choice([0.1,0.03]),#0.001]),
            'num_leaves': np.random.choice([15,31]),#,61,127]),
            'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
            'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
            'min_data_in_leaf': np.random.randint(20,50),  #minimal number of data in one leaf. 
            'feature_fraction': np.random.randint(9,11)/10,
            'feature_fraction_seed': seed,
            'early_stopping_round':10,
            'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
            'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
            'bagging_seed': seed,
            'scale_pos_weight': 1,
            'metric': 'auc'
#             'objective': 'binary',
#             'boosting': 'gbdt',
#             'num_boost_round': 10, # ignored in params. extract it and put it in input arguments in train or cv explicitly
#                                    # seems to work fine as the upper limit when combined with early_stopping_round
#             'learning_rate': 0.01,
#             'num_leaves': 50,
#             'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
#             'max_depth': 5, # no limit. This is used to deal with over-fitting when #data is small.
#             'min_split_gain': 0.02,
#             'min_child_weight': 18.6,
#             'feature_fraction': 0.1,
#             'feature_fraction_seed': 1992,
#             'early_stopping_round':10,
#             'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
#             'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
#             'bagging_seed': 1992,
#             'scale_pos_weight': 1,
#             'lambda_l1': 0.8,
#             'lambda_l2': 0.6,
#             'metric': 'auc'
        }
    elif model == 'xgb':
        params = {
            'eta': 0.1,
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'booster': 'gbtree',
            'tree_method': 'hist',
            'max_bin': 8,
            'scale_pos_weight': 1,
            'seed': seed,
            'nthread': 10,
            'max_depth': 6,
            'min_child_weight': 22,
            'gamma': 0,
            'subsample': 0.7,
            'colsample_bytree': 0.7,
            'colsample_bylevel': 0.675,
            'alpha': 0,
            'lambda': 0,
            'num_boost_round': 50,
            'early_stopping_rounds': 1
        }
    elif model == 'catb':
        params = {
            'iterations': 30,
            'depth': np.random.randint(6,11),
            'l2_leaf_reg': np.random.randint(0, 31)/10,
        #     'custom_metric': 'AUC',
            'eval_metric': 'AUC',
            'random_seed': seed,
            'logging_level': 'Verbose',
            'thread_count': 15
        }
    elif model == 'nn':
        params = {
            'nn_seed': int(time.time() * 1000000) % 45234634,
            'ep_for_lr': np.random.randint(2,10),
            'lr_init': 0.01,
            'lr_fin': np.random.randint(1,5)/1000,  # if == lr_init, then no lr decay
            'batch_size': np.random.choice([128, 256, 512, 1024]),
            "pred_batch_size": 5000,
            'max_ep': 100,
            'patience': 10, #np.random.randint(10, 25),
            'cat_emb_outdim': 30, # could be a constant or a dict (col name:embed out dim). e.g.:
                                  # embed_outdim = [3, 3, 8, 8, 3]
                                  # embed_outdim_dict = dict(zip(X_train.columns.values, embed_outdim))
                                  # then assige embed_outdim_dict to cat_emb_outdim
            'cat_emb_drop_rate': np.random.randint(1,4)/10, 
            'num_layers_dense_units': [1000, 500, 100],
            'num_layers_drop_rate': np.random.randint(2,6)/10,
            'combined_layers_dense_units': [100, 50],
            'combined_layers_drop_rate': np.random.randint(1,3)/10,
            'monitor': 'val_auc',  # or val_loss (MUST HAVE)
            'mode': 'max',  # MUST HAVE
            'int_list': ['num_layers_dense_units', 'combined_layers_dense_units']
        }
    elif model == 'stacknet_layer2_nn':
        params = {
            'nn_seed': int(time.time() * 1000000) % 45234634,
            'ep_for_lr': 1,
            'lr_init': 0.01,
            'lr_fin': 0.01, # if == lr_init, then no lr decay
            'batch_size': 128,
            "pred_batch_size": 50000,
            'best_epoch': 1,
            'patience': 1,
            'categorical_feature': [],
            'cat_emb_outdim': 50, # could be a constant or a dict (col name:embed out dim). e.g.:
                                  # embed_outdim = [3, 3, 8, 8, 3]
                                  # embed_outdim_dict = dict(zip(X_train.columns.values, embed_outdim))
            'num_layers_dense_units': [],
            'combined_layers_dense_units': [10, 5],
            'combined_layers_drop_rate': 0,
            'monitor': 'val_auc',  # or val_loss (MUST HAVE)
            'mode': 'max',  # MUST HAVE
            'int_list': ['num_layers_dense_units', 'combined_layers_dense_units']
        }
    return params, seed

In [9]:
p, s = params_gen('stacknet_layer2_nn')
p, s

({'batch_size': 128,
  'best_epoch': 1,
  'cat_emb_outdim': 50,
  'categorical_feature': [],
  'combined_layers_dense_units': [10, 5],
  'combined_layers_drop_rate': 0,
  'ep_for_lr': 1,
  'int_list': ['num_layers_dense_units', 'combined_layers_dense_units'],
  'lr_fin': 0.01,
  'lr_init': 0.01,
  'mode': 'max',
  'monitor': 'val_auc',
  'nn_seed': 3332108,
  'num_layers_dense_units': [],
  'patience': 1,
  'pred_batch_size': 50000},
 3332098)

In [10]:
import sys
import time

class Logger(object):
    def __init__(self, logtofile=True, logfilename='log'):
        self.terminal = sys.stdout
        self.logfile = "{}_{}.log".format(logfilename, int(time.time()))
        self.logtofile = logtofile

    def write(self, message):
#         self.terminal.write(message)
        if self.logtofile:
            self.log = open(self.logfile, "a")
            self.log.write('['+utils.get_time()+'] '+message)  
            self.log.close()


    def flush(self):
        #this flush method is needed for python 3 compatibility.
        #this handles the flush command by doing nothing.
        #you might want to specify some extra behavior here.
        pass    

def divert_printout_to_file():
    sys.stdout = Logger(logfilename='logfile')

In [11]:
# divert_printout_to_file()

In [12]:
from automl_app.app import AlphaBoosting

automl_config_file = project_path + 'automl_config.json'
run_record_file_name = project_path + 'last_run_record.json' # don't created this file
AlphaBoosting(automl_config_file, features_to_gen, params_gen)

2018-07-18 15:22:46 | INFO    | automl_app.app.AlphaBoosting.__init__             | #23  | ==========START==========
2018-07-18 15:22:47 | INFO    | automl_app.app.AlphaBoosting.__init__             | #76  | Validation is randomly chosen. Percentage: 15.0%, Stratify: True


> /home/kai/data/shiyi/AlphaBoosting/automl/automl_app/app.py(78)__init__()
-> del X, y, X_train, X_validation, y_train, y_validation; gc.collect()
(Pdb) len(self.validation_index)
150000
(Pdb) self.validation_index[:10]
[452966, 189988, 641855, 838926, 167219, 258467, 837718, 807658, 978001, 83707]
(Pdb) y_train.value_counts()[0]/len(y_train)
0.8070176470588235
(Pdb) y_validation.value_counts()[0]/len(y_validation)
0.80702
(Pdb) n
> /home/kai/data/shiyi/AlphaBoosting/automl/automl_app/app.py(82)__init__()
-> downsampling_amount_changed = False
(Pdb) self.validation_index[:10]
[452966, 189988, 641855, 838926, 167219, 258467, 837718, 807658, 978001, 83707]
(Pdb) c


2018-07-18 15:24:26 | INFO    | automl_app.app.AlphaBoosting.__init__             | #82  | Run record file [./output/last_run_record.json] not found. Begin the first time run...
2018-07-18 15:24:26 | INFO    | automl_app.app.AlphaBoosting.__init__             | #82  | generate todo list
2018-07-18 15:24:26 | INFO    | automl_app.app.AlphaBoosting.__init__             | #82  | STAGE: FEATURE_ENGINEERING
2018-07-18 15:24:26 | INFO    | automl_app.app.AlphaBoosting.__init__             | #82  | STAGE: VALIDATION_DOWNSAMPLING_GEN
2018-07-18 15:24:26 | INFO    | automl_app.app.AlphaBoosting.__init__             | #82  | STAGE: CONCAT_DATA
2018-07-18 15:24:26 | INFO    | automl_app.app.AlphaBoosting.__init__             | #82  | STAGE: GRID_SEARCH
2018-07-18 15:24:26 | INFO    | automl_app.app.AlphaBoosting._get_final_data      | #359 | DEBUG mode is on, 5.0% of train,val data are chosen
2018-07-18 15:24:26 | INFO    | automl_app.app.AlphaBoosting._get_final_data      | #368 | Data <ordinal2

[0]	train-auc:0.700482+0.00160358	test-auc:0.689818+0.00369153
[1]	train-auc:0.70719+0.0015167	test-auc:0.692853+0.00477045
[2]	train-auc:0.710007+0.00381135	test-auc:0.694326+0.00664052
[3]	train-auc:0.714543+0.00187389	test-auc:0.699509+0.00396955
[4]	train-auc:0.71667+0.00269235	test-auc:0.700713+0.003999
[5]	train-auc:0.718804+0.00231821	test-auc:0.702473+0.0032154
[6]	train-auc:0.720125+0.00198081	test-auc:0.703048+0.00396003
[7]	train-auc:0.721106+0.001879	test-auc:0.703958+0.00405519
[8]	train-auc:0.722483+0.0014404	test-auc:0.705569+0.0037824
[9]	train-auc:0.723598+0.000633838	test-auc:0.706545+0.00372993
[10]	train-auc:0.724992+0.000910472	test-auc:0.706967+0.0038864
[11]	train-auc:0.725973+0.000763211	test-auc:0.707077+0.00409575
[12]	train-auc:0.727288+0.000638136	test-auc:0.707664+0.00400741
[13]	train-auc:0.728589+0.000597385	test-auc:0.708549+0.00425218
[14]	train-auc:0.729803+0.000871998	test-auc:0.709186+0.00415271
[15]	train-auc:0.73096+0.000652821	test-auc:0.709418+0.

2018-07-18 15:24:29 | INFO    | automl_libs.grid_search._xgb_gs              | #321 | val_auc: 0.72181 | train_auc: 0.76034 (cv)
2018-07-18 15:24:29 | INFO    | automl_libs.grid_search._xgb_gs              | #343 | [do_preds] is True, generating predictions ...
2018-07-18 15:24:29 | INFO    | automl_libs.grid_search._xgb_gs              | #345 | Retrain model using best_round [46] and all data...


...

2018-07-18 15:24:36 | INFO    | automl_libs.base_layer_utils.get_oof              | #449 | metric of fold 4: 0.7267806743266345
2018-07-18 15:24:36 | INFO    | automl_libs.base_layer_utils.get_oof              | #435 | processing fold 5 of 5...
2018-07-18 15:24:36 | INFO    | automl_libs.stack_layer_estimator.XgboostBLE.train                | #208 | No evaluation set, thus not possible to use early stopping. Please train with your best params.


[0]	train-auc:0.704337
[5]	train-auc:0.719691
[10]	train-auc:0.726798
[15]	train-auc:0.731906
[20]	train-auc:0.73689
[25]	train-auc:0.741594
[30]	train-auc:0.746731
[35]	train-auc:0.752022
[40]	train-auc:0.75662
[45]	train-auc:0.761392
[49]	train-auc:0.763579


2018-07-18 15:24:36 | INFO    | automl_libs.base_layer_utils.get_oof              | #449 | metric of fold 5: 0.7195589038079377
2018-07-18 15:24:36 | INFO    | automl_libs.base_layer_utils.compute_layer1_oof   | #528 | oof is done
2018-07-18 15:24:36 | INFO    | automl_libs.base_layer_utils.compute_layer1_oof   | #532 | Training using all data and gen prediction for submission...
2018-07-18 15:24:36 | INFO    | automl_libs.stack_layer_estimator.XgboostBLE.train                | #208 | No evaluation set, thus not possible to use early stopping. Please train with your best params.


[0]	train-auc:0.703293
[5]	train-auc:0.719287
[10]	train-auc:0.725151
[15]	train-auc:0.728777
[20]	train-auc:0.734332
[25]	train-auc:0.739297
[30]	train-auc:0.743274
[35]	train-auc:0.747879
[40]	train-auc:0.752416
[45]	train-auc:0.75643
[49]	train-auc:0.760564


2018-07-18 15:24:37 | INFO    | automl_libs.base_layer_utils.BaseLayerResultsRepo.update_report        | #280 | StackNet report updated: apB6__XGB_ordinal2_layer1: oof_cv_score => 0.7233451035300688
2018-07-18 15:24:37 | INFO    | automl_libs.base_layer_utils.BaseLayerResultsRepo.add_score            | #299 | apB6__XGB_ordinal2_layer1 found in repo. Update score from 0 to 0.72260
2018-07-18 15:24:37 | INFO    | automl_libs.base_layer_utils.BaseLayerResultsRepo.update_report        | #280 | StackNet report updated: apB6__XGB_ordinal2_layer1: gs_val_auc => 0.7225952000000001
2018-07-18 15:24:37 | INFO    | automl_libs.base_layer_utils.BaseLayerResultsRepo.update_report        | #280 | StackNet report updated: apB6__XGB_ordinal2_layer1: test_score => 0.7147251478958567
2018-07-18 15:24:37 | INFO    | automl_libs.base_layer_utils.BaseLayerResultsRepo.save                 | #393 | StackNet data saved for: ['IJfZ__XGB_ordinal2_layer1', 'apB6__XGB_ordinal2_layer1']
2018-07-18 15:24:37 | INFO 